<a href="https://colab.research.google.com/github/0nlyDust/Wumpus/blob/main/Wumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Solución para el juego del Wumpus.
Version: 1.0
Autor: María Molina Goyena
Descripción: Este código simula un entorno 2D donde un agente explora un tablero para encontrar oro, evitar hoyos
y morir por enfrentarse al Wumpus. Incluye funcionalidades para generar el tablero, mover al agente, manejar estados, y calcular
el costo de movimientos.

Requisitos:
- Python 3.6+
- IPython para visualización en Jupyter Notebook
- Librerías estándar: random, math, time, itertools, etc.
"""

# Importación de librerias y módulos necesarias
from typing import Tuple, List, Optional
from IPython.display import HTML, display, clear_output
from collections import Counter
from random import randint
from copy import deepcopy
import itertools
import random
import math
import time
import copy
import os

# Variables globales
equivalencias = {}  # Mapeo entre notaciones del tablero y valores numéricos
new_visited_positions = {}  # Controla las posiciones visitadas para penalizar movimientos repetidos
count_hoyos = 0  # Cuenta los pozos generados en el tablero





In [2]:
from google.colab import files
uploaded = files.upload()

!unzip ImagenesCasillas.zip -d Wumpus


class Tablerowumpus:
    """
    Clase que representa el tablero del mundo del Wumpus.

    Atributos:
    ----------
    - matrix: Matriz 2D que representa el tablero.
    - agente: Posición del agente en el tablero.
    - oro: Posición del oro en el tablero.
    - wumpus: Posición del Wumpus en el tablero.
    - hoyos: Lista de posiciones de hoyos en el tablero.

    Métodos principales:
    ---------------------
    - mover_tile: Modifica los valores de las celdas del tablero.
    - obtener_vecinos: Devuelve las posiciones adyacentes a una celda.
    - utility: Calcula la utilidad de un movimiento.
    """

    def __init__(self, matrix, num_hoyos: int, coord_oro: Tuple[int, int], coord_wumpus: Tuple[int, int]):
        """
        Versión: 1.0
        Descripción:
            Inicializa el tablero del mundo del Wumpus, configurando los objetos como el agente, el oro, el Wumpus, los hoyos, etc.

        Parámetros de entrada:
            - matrix (List[List[int]]): Matriz que representa el tablero del mundo.
            - num_hoyos (int): Número de hoyos a colocar en el tablero.
            - coord_oro (Tuple[int, int]): Coordenadas donde se colocará el oro.
            - coord_wumpus (Tuple[int, int]): Coordenadas donde se colocará el Wumpus.

        Salida:
            Ninguno. Modifica el estado interno del tablero.

        Notas:
            - Si las coordenadas del oro o Wumpus no son proporcionadas, se generan aleatoriamente.
        """
        self.setMatrix(matrix)
        self.agente = (len(self.matrix) - 1, 0)  # Inicialmente, el agente está en la esquina inferior izquierda.
        self.mover_tile(self.agente[0], self.agente[1], 1)  # Coloca al agente en el tablero.
        vecinos_agente = self.obtener_vecinos(self.agente)

        # Configuración de la posición del oro
        self.oro = None
        if coord_oro == (None, None):
            while (
                self.oro is None or
                self.oro in vecinos_agente or
                self.oro == (6, 1) or
                self.oro == (7, 0) or
                self.oro == self.agente
            ):
                self.oro = self.generar_posicion_aleatoria()
        else:
            self.oro = coord_oro

        if self.oro is not None:
            self.mover_tile(self.oro[0], self.oro[1], 4)  # Coloca el oro en el tablero.

        # Configuración de la posición del Wumpus
        self.wumpus = None
        if coord_wumpus == (None, None):
            while (
                self.wumpus is None or
                self.wumpus == self.oro or
                self.wumpus in vecinos_agente or
                self.wumpus == (6, 1) or
                self.wumpus == self.agente
            ):
                self.wumpus = self.generar_posicion_aleatoria()
        else:
            self.wumpus = coord_wumpus

        if self.wumpus is not None:
            self.mover_tile(self.wumpus[0], self.wumpus[1], 2)  # Coloca el Wumpus en el tablero.

        # Generación de "hedor" en casillas adyacentes al Wumpus
        vecinos_wumpus = self.obtener_vecinos(self.wumpus)
        for vecino in vecinos_wumpus:
            self.mover_tile(vecino[0], vecino[1], 3)

        # Coloca hoyos en el tablero
        self.colocar_hoyos(num_hoyos or random.randint(3, 5))

    def get_content(self, coord):
        """
        Versión: 1.0
        Descripción:
            Devuelve el contenido textual de una celda en función de su valor numérico.

        Parámetros de entrada:
            - coord (Tuple[int, int]): Coordenada de la casilla a consultar.

        Salida:
            - str: Descripción de la casilla correspondiente.

        Notas:
            - Usa un diccionario predefinido para traducir valores numéricos en texto descriptivo.
        """
        contenidos = {
            0: "CasillaVacia", 1: "CasillaAgente", 2: "CasillaWumpus",
            3: "CasillaHedor", 4: "CasillaOro", 5: "CasillaBrisa",
            6: "CasillaHueco", 7: "CasillaAgenteHueco", 8: "CasillaAgenteHuecoBrisa",
            9: "CasillaHedorBrisa", 10: "CasillaAgenteFlecha", 11: "CasillaBrisaOro",
            12: "CasillaWumpusFlecha", 13: "CasillaHedorOro", 14: "CasillaHedorBrisaOro",
            15: "CasillaBrisaHueco", 16: "CasillaAgenteWumpus", 17: "CasillaAgenteBrisa",
            18: "CasillaAgenteHedor", 19: "CasillaAgenteOro", 20: "CasillaAgenteBrisaOro",
            21: "CasillaAgenteHedorOro", 22: "CasillaAgenteHedorBrisaOro", 23: "CasillaAgenteHedorBrisa",
            24: "CasillaFlecha", 25: "CasillaWumpusFlecha"
        }
        # Devuelve la descripción de la casilla o "CasillaVacia" si no hay coincidencia
        return contenidos.get(self.matrix[coord[0]][coord[1]], "CasillaVacia")


    def get_html(self):
        """
        Versión: 1.0
        Descripción:
            Genera el código HTML necesario para representar el tablero visualmente, usando imágenes correspondientes a cada tipo de casilla.

        Parámetros de entrada:
            Ninguno.

        Salida:
            - HTML: Devuelve una representación en HTML del tablero de juego.

        Notas:
            - Utiliza imágenes en función del tipo de casilla para representar visualmente el tablero.
        """
        element_image = {
            "CasillaVacia": "./ImagenesCasillas/0.png",
            "CasillaAgente": "./ImagenesCasillas/1.png",
            "CasillaWumpus": "./ImagenesCasillas/2.png",
            "CasillaHedor": "./ImagenesCasillas/3.png",
            "CasillaOro": "./ImagenesCasillas/4.png",
            "CasillaBrisa": "./ImagenesCasillas/5.png",
            "CasillaHueco": "./ImagenesCasillas/6.png",
            "CasillaAgenteHueco": "./ImagenesCasillas/7.png",
            "CasillaAgenteHuecoBrisa": "./ImagenesCasillas/8.png",
            "CasillaHedorBrisa": "./ImagenesCasillas/9.png",
            "CasillaAgenteFlecha": "./ImagenesCasillas/10.png",
            "CasillaBrisaOro": "./ImagenesCasillas/11.png",
            "CasillaWumpusFlecha": "./ImagenesCasillas/12.png",
            "CasillaHedorOro": "./ImagenesCasillas/13.png",
            "CasillaHedorBrisaOro": "./ImagenesCasillas/14.png",
            "CasillaBrisaHueco": "./ImagenesCasillas/15.png",
            "CasillaAgenteWumpus": "./ImagenesCasillas/16.png",
            "CasillaAgenteBrisa": "./ImagenesCasillas/17.png",
            "CasillaAgenteHedor": "./ImagenesCasillas/18.png",
            "CasillaAgenteOro": "./ImagenesCasillas/19.png",
            "CasillaAgenteBrisaOro": "./ImagenesCasillas/20.png",
            "CasillaAgenteHedorOro": "./ImagenesCasillas/21.png",
            "CasillaAgenteHedorBrisaOro": "./ImagenesCasillas/22.png",
            "CasillaAgenteHedorBrisa": "./ImagenesCasillas/23.png",
            "CasillaFlecha": "./ImagenesCasillas/24.png"
        }

        height = len(self.matrix)  # Altura del tablero
        width = len(self.matrix[0])  # Ancho del tablero

        # Estilo del HTML para el tablero
        html_string = """
            <style>
            #game-table {
                width: auto;
                background-color: #ffffff !important;
            }
            #game-table td {
                border: 2px solid black !important;
                text-align: center;
                width: 75px;
                height: 75px;
            }
            </style>
            <table id="game-table">
        """

        new_row = "<tr>"  # Inicio de fila
        end_row = "</tr>"  # Fin de fila

        # Construcción de las filas y columnas con sus respectivas imágenes
        for i in range(height):
            html_string += new_row
            for j in range(width):
                content = self.get_content((i, j))  # Contenido de la casilla
                drawing = element_image.get(content, "./ImagenesCasillas/0.png")  # Imagen asociada
                html = '<td><img class="game" src="%s" alt=""></img></td>' % drawing
                html_string += html
            html_string += end_row
        html_string += "</table>"

        return HTML(html_string)  # Devuelve el HTML como salida

    #def mostrar_tablero(self):
        """
        Versión: 1.0
        Descripción:
            Muestra el tablero de juego llamando al método `get_html` para generar una representación visual del tablero en formato HTML.

        Parámetros de entrada:
            Ninguno.

        Salida:
            HTML: Devuelve una representación visual del tablero en formato HTML.

        Notas:
            - Utiliza el método `get_html` para generar el HTML y luego lo pasa a través de la clase `HTML` para mostrarlo.
        """
        #return HTML(self.get_html())


    import webbrowser

    def mostrar_tablero(tablero):
        """
        Muestra el tablero en consola o abre un HTML en el navegador.
        """
        # Opción consola
        try:
            print(tablero)  # Asegúrate de implementar __str__ en la clase Tablerowumpus
        except:
             # Si no hay __str__, generamos archivo HTML
            html = tablero.get_html()
            with open("tablero.html", "w", encoding="utf-8") as f:
                f.write(html)
            webbrowser.open("tablero.html")


    def colocar_hoyos(self, num_hoyos: int):
        """
        Versión: 1.0
        Descripción:
            Coloca hoyos en el tablero, asegurándose de que no estén en posiciones inválidas (como donde está el agente o el Wumpus).

        Parámetros de entrada:
            - num_hoyos (int): Número de hoyos a colocar en el tablero.

        Salida:
            Ninguno. Modifica directamente el contenido de la matriz del tablero.

        Notas:
            - Los hoyos no pueden colocarse en las posiciones ocupadas por el agente, el Wumpus, ni el oro.
        """
        self.hoyos = []
        vecinos_wumpus = self.obtener_vecinos(self.wumpus)  # Vecinos del Wumpus para evitar conflictos

        while len(self.hoyos) < num_hoyos:
            hoyo = self.generar_posicion_aleatoria()  # Generar posición aleatoria
            if hoyo != self.wumpus and hoyo not in vecinos_wumpus and hoyo != self.agente and hoyo != self.oro:
                self.mover_tile(hoyo[0], hoyo[1], 6)  # Colocar un hoyo
                vecinos_hoyos = self.obtener_vecinos(hoyo)  # Vecinos del hoyo
                for vecino in vecinos_hoyos:
                    self.mover_tile(vecino[0], vecino[1], 5)  # Colocar brisas alrededor
                self.hoyos.append(hoyo)

    def obtener_vecinos(self, coord: Tuple[int, int]) -> List[Tuple[int, int]]:
        """
        Versión: 1.0
        Descripción:
            Devuelve una lista de las coordenadas de las casillas vecinas (adyacentes) a una posición dada.

        Parámetros de entrada:
            - pos (Tuple[int, int]): Coordenada (fila, columna) de la casilla central.

        Salida:
            - List[Tuple[int, int]]: Lista de coordenadas de las casillas vecinas.

        Notas:
            - Considera solo las casillas adyacentes en las 4 direcciones cardinales (arriba, abajo, izquierda, derecha).
            - Si una casilla está fuera del límite del tablero, no se incluye como vecina.
        """
        row, col = coord
        vecinos = []
        if row > 0:
            vecinos.append((row - 1, col))
        if row < len(self.matrix) - 1:
            vecinos.append((row + 1, col))
        if col > 0:
            vecinos.append((row, col - 1))
        if col < len(self.matrix[0]) - 1:
            vecinos.append((row, col + 1))
        return vecinos

    def actualizar_celda(self, current_tile: int, new_tile: int) -> int:
        """
        Versión: 1.0
        Descripción:
            Combina dos elementos en una celda según las reglas definidas en un diccionario de combinaciones, devolviendo el nuevo valor que resulta de la combinación.

        Parámetros de entrada:
            current_tile (int): El valor actual de la celda.
            new_tile (int): El valor que se intenta combinar con la celda actual.

        Salida:
            int: El nuevo valor de la celda después de la combinación.

        Notas:
            - La función usa un diccionario de combinaciones para determinar cómo se combinan los elementos de las celdas.
            - Si no existe una combinación definida entre los dos valores, la función devuelve el valor `new_tile` como resultado.
        """
        combinaciones = {
            1: {2: 16, 3: 18, 4: 19, 5: 17, 6: 7, 9: 23, 11: 20, 13: 21, 14: 22, 24: 10},  #A
            2: {1: 16, 5: 2,6: 2,24: 25},  #W
            3: {1: 18, 4: 13,5: 9, 11: 14, 21: 22, 17: 23}, #H
            4: {1: 19, 3: 13, 5: 11,6: 4, 9: 14, 17: 20, 18: 21, 23: 22},  #O
            5: {1: 17,3: 9, 4: 11, 5: 5,6: 15, 13: 14, 18: 23, 19: 20, 21: 22}, #B
            6: {1: 7, 3: 12, 5: 15, 15: 15}, #C
            7: {5: 8}, #AC
            8: {}, #CBA
            9: {4: 14, 5: 9, 19: 22, 1: 23}, #HB
            10: {},
            11: {3: 14,5: 11, 1: 20, 18: 22}, #BO
            12: {}, #WD
            13: {5: 14, 1: 21, 17: 22}, #HO
            14: {1: 22}, #HBO
            15: {1: 8}, #CB
            16: {},
            17: {6: 8, 5: 17, 4: 20, 13: 22, 3: 23}, #AB
            18: {4: 21, 5: 23}, #AH
            19: {5: 20, 3: 21, 9: 22}, #AO
            20: {3: 22}, #ABO
            21: {5: 22}, #AHO
            22: {5: 26}, #AHBO
            23: {4: 22, 8: 36}, # AHB
            24: {1: 10, 2: 25}, #D
        }
        # Obtiene la combinación o retorna el nuevo elemento si no hay combinación
        tile = combinaciones.get(current_tile, {}).get(new_tile, new_tile)
        return tile


    def generar_posicion_aleatoria(self):
        """
        Versión: 1.0
        Descripción:
            Genera una posición aleatoria dentro de los límites del tablero.

        Parámetros de entrada:
            Ninguno.

        Salida:
            - Tuple[int, int]: Coordenada aleatoria (fila, columna).

        Notas:
            - La posición generada es completamente aleatoria, sin restricciones adicionales.
        """
        row = random.randint(0, len(self.matrix) - 1)
        col = random.randint(0, len(self.matrix) - 1)
        return (row, col)

    def get_hoyo(self) -> List[Tuple[int, int]]:
        """
        Versión: 1.0
        Descripción:
            Busca y devuelve las coordenadas de las celdas que contienen un hoyo, identificadas por los valores 6, 7, 8, o 15 en la matriz.

        Parámetros de entrada:
            Ninguno.

        Salida:
            List[Tuple[int, int]]: Lista de tuplas con las coordenadas (i, j) de las celdas que contienen hoyos.

        Notas:
            - Recorre toda la matriz `self.matrix` y busca celdas cuyo valor esté en el conjunto `valores_buscados` (6, 7, 8, 15).
            - La función devuelve una lista de las coordenadas donde se encuentran estos valores.
        """
        valores_buscados = {6, 7, 8, 15}
        coordenadas = []
        coord_filtradas = []
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[i])):
                if self.matrix[i][j] in valores_buscados:
                    coordenadas.append((i, j))
        return coordenadas

    def resetear_celda(self, current_tile: int, new_tile: int) -> int:
        """
        Versión: 1.0
        Descripción:
            Resetea el valor de una celda según las combinaciones predefinidas en un diccionario. Si no hay una combinación definida, devuelve el valor original de la celda.

        Parámetros de entrada:
            current_tile (int): El valor actual de la celda.
            new_tile (int): El valor con el que se intenta resetear la celda.

        Salida:
            int: El nuevo valor de la celda después de aplicar la combinación o el valor original si no hay combinación.

        Notas:
            - Utiliza un diccionario `combinaciones` para determinar cómo deben combinarse los valores de las celdas.
            - Si no existe una combinación para los valores proporcionados, el valor de `new_tile` se devuelve sin cambios.
        """
        combinaciones = {
            1: {1: 0},
            2: {},
            3: {},
            4: {},
            5: {5: 0},
            6: {6: 0},
            7: {1: 6, 6: 1},
            8: {1: 15, 5: 7,6: 17},
            9: {5: 3, 3: 5},
            10: {},
            11: {5: 4},
            12: {},
            13: {},
            14: {5: 13},
            15: {6: 5, 5: 6},
            16: {},
            17: {1: 5, 5: 1},
            18: {1: 3},
            19: {1: 4},
            20: {1: 11, 5: 19},
            21: {1: 13, 5: 18},
            22: {1: 14, 5: 21},
            23: {1: 9, 5: 22},
            24: {}
        }
        # Obtiene la combinación o retorna el nuevo elemento si no hay combinación
        tile = combinaciones.get(current_tile, {}).get(new_tile, new_tile)
        return tile

    def __eq__(self, other) -> bool:
        """
        Versión: 1.0
        Descripción:
            Compara dos matrices para determinar si son iguales. La comparación se hace verificando que las matrices tengan el mismo tamaño y sus elementos sean idénticos.

        Parámetros de entrada:
            other (objeto): Otro objeto de tipo similar al actual con una matriz `matrix` que se va a comparar.

        Salida:
            bool: Retorna `True` si las matrices son iguales, de lo contrario `False`.

        Notas:
            - Compara las dimensiones de las matrices y luego compara cada elemento en las posiciones correspondientes.
        """
        if len(self.matrix) == len(other.matrix) and len(self.matrix[0]) == len(other.matrix[0]):
            for i in range(len(self.matrix)):
                for j in range(len(self.matrix[0])):
                    if self.matrix[i][j] != other.matrix[i][j]:
                        return False
            return True
        return False

    def setMatrix(self, matrix):
        """
        Versión: 1.0
        Descripción:
            Establece una nueva matriz como la matriz interna del objeto.

        Parámetros de entrada:
            matrix (List[List]): La nueva matriz que se asignará a la propiedad `self.matrix`.

        Salida:
            Ninguna.

        Notas:
            - La matriz se establece directamente en el atributo `self.matrix`.
        """
        self.matrix = deepcopy(matrix)

    def getMatrix(self) -> List[List]:
        """
        Versión: 1.0
        Descripción:
            Devuelve una copia profunda de la matriz interna del objeto.

        Parámetros de entrada:
            Ninguno.

        Salida:
            List[List]: Una copia de la matriz interna `self.matrix`.

        Notas:
            - Se utiliza `deepcopy` para evitar que se modifique la matriz interna desde fuera del objeto.
        """
        return deepcopy(self.matrix)

    def print_matrix(self) -> List[List[int]]:
        """
        Versión: 1.0
        Descripción:
            Imprime la matriz interna del objeto fila por fila.

        Parámetros de entrada:
            Ninguno.

        Salida:
            List[List[int]]: La matriz interna `self.matrix` impresa fila por fila.

        Notas:
            - Esta función se utiliza solo para mostrar la matriz en un formato legible en la consola.
        """
        for row in self.getMatrix():
            print(row)

    def utility(self, coord1: Tuple[int, int], coord2: Tuple[int, int]) -> float:
        """
        Versión: 1.0
        Descripción:
            Calcula la utilidad de un movimiento entre dos coordenadas, considerando factores como la distancia, penalizaciones por caer en un hoyo, estar en una brisa, estar en un hedor, o retroceder a una posición previamente visitada.

        Parámetros de entrada:
            coord1 (Tuple[int, int]): Las coordenadas de la celda actual.
            coord2 (Tuple[int, int]): Las coordenadas de la celda objetivo.

        Salida:
            float: El coste total del movimiento, considerando penalizaciones.

        Notas:
            - La función calcula la distancia entre las dos celdas y ajusta el coste basado en diferentes penalizaciones.
        """
        global new_visited_positions
        x1, y1 = coord1
        x2, y2 = coord2
        distancia = 1 / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) + 0.02)

        penal_brisa = penal_hoyo = penal_hedor = penal_retroceso = coste_total = 0

        # Penalización si el agente cae en un hoyo
        if self.matrix[coord1[0]][coord1[1]] == 7 or self.matrix[coord1[0]][coord1[1]] == 8:
            penal_hoyo = distancia / 2

        # Penalización si el agente está en una brisa
        if self.matrix[coord1[0]][coord1[1]] in (17, 20, 22, 23):
            penal_brisa = distancia / 6

        # Penalización si el agente está en un hedor
        if self.matrix[coord1[0]][coord1[1]] in (18, 21, 22, 23):
            penal_hedor = distancia / 3

        # Penalización por retroceso
        if self.mov_repetido(coord1):
            penal_retroceso = 1

        coste_total = distancia - penal_hoyo - penal_hedor - penal_brisa - penal_retroceso
        return coste_total

    def mov_repetido(self, coord: Tuple[int,int]) -> bool:
        """
        Versión: 1.0
        Descripción:
            Determina si una coordenada ha sido visitada previamente durante el juego.

        Parámetros de entrada:
            coord (Tuple[int, int]): La coordenada que se desea verificar.

        Salida:
            bool: `True` si la coordenada ya ha sido visitada, `False` en caso contrario.

        Notas:
            - Usa una lista global `new_visited_positions` para almacenar las coordenadas visitadas.
        """
        global new_visited_positions
        if coord in new_visited_positions:
            return True
        return False

    def canMoveUp(self, row: int, col: int) -> bool:
        """
        Versión: 1.0
        Descripción:
            Determina si el agente puede moverse hacia arriba desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del agente.
            - col (int): Índice de columna actual del agente.

        Salida:
            - bool: True si el movimiento hacia arriba es posible, False en caso contrario.

        Notas:
            - Un movimiento hacia arriba es válido si la fila actual es mayor a 0.
        """
        return row > 0

    def canMoveDown(self, row: int, col: int) -> bool:
        """
        Versión: 1.0
        Descripción:
            Determina si el agente puede moverse hacia abajo desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del agente.
            - col (int): Índice de columna actual del agente.

        Salida:
            - bool: True si el movimiento hacia abajo es posible, False en caso contrario.

        Notas:
            - Un movimiento hacia abajo es válido si la fila actual es menor que la longitud de la matriz - 1.
        """
        return row < len(self.matrix) - 1

    def canMoveLeft(self, row: int, col: int) -> bool:
        """
        Versión: 1.0
        Descripción:
            Determina si el agente puede moverse hacia la izquierda desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del agente.
            - col (int): Índice de columna actual del agente.

        Salida:
            - bool: True si el movimiento hacia la izquierda es posible, False en caso contrario.

        Notas:
            - Un movimiento hacia la izquierda es válido si la columna actual es mayor a 0.
        """
        return col < len(self.matrix) - 1

    def canMoveRight(self, row: int, col: int) -> bool:
        """
        Versión: 1.0
        Descripción:
            Determina si el agente puede moverse hacia la derecha desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del agente.
            - col (int): Índice de columna actual del agente.

        Salida:
            - bool: True si el movimiento hacia la derecha es posible, False en caso contrario.

        Notas:
            - Un movimiento hacia la derecha es válido si la columna actual es menor que la longitud de la fila - 1.
        """
        return col > 0

    def getAvailableMovesForMax(self, row: int, col: int) -> List[Tuple[int, int]]:
        """
        Versión: 1.0
        Descripción:
            Obtiene todos los movimientos válidos que puede realizar el jugador 'Max' desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del jugador 'Max'.
            - col (int): Índice de columna actual del jugador 'Max'.

        Salida:
            - List[Tuple[int, int]]: Lista de coordenadas de las casillas a las que puede moverse.

        Notas:
            - Considera movimientos hacia arriba, abajo, izquierda y derecha.
        """
        moves = []
        if self.canMoveDown(row, col):
            moves.append((row + 1, col))
        if self.canMoveLeft(row, col):
            moves.append((row, col + 1))
        if self.canMoveRight(row, col):
            moves.append((row, col - 1))
        if self.canMoveUp(row, col):
            moves.append((row - 1, col))
        return moves

    def getAvailableMovesForMin(self, row: int, col: int) -> List[Tuple[int, int]]:
        """
        Versión: 1.0
        Descripción:
            Obtiene todos los movimientos válidos que puede realizar el jugador 'Min' desde la posición dada.

        Parámetros de entrada:
            - row (int): Índice de fila actual del jugador 'Min'.
            - col (int): Índice de columna actual del jugador 'Min'.

        Salida:
            - List[Tuple[int, int]]: Lista de coordenadas de las casillas a las que puede moverse.

        Notas:
            - Reutiliza la lógica de movimientos de `getAvailableMovesForMax`.
        """
        return self.getAvailableMovesForMax(row, col)

    def moveCanBeMade(self, player: int, coord: Tuple[int, int]) -> bool:
        """
        Versión: 1.0
        Descripción:
            Verifica si el jugador puede realizar un movimiento desde la posición actual.

        Parámetros de entrada:
            - player (int): Identificador del jugador (1 para Max, 2 para Min).
            - coord (Tuple[int, int]): Coordenadas de la posición actual.

        Salida:
            - bool: True si el jugador puede realizar al menos un movimiento válido, False en caso contrario.
        """
        x, y = coord
        available_moves = self.getAvailableMovesForMax(x, y) if player == 1 else self.getAvailableMovesForMin(x, y)
        return len(available_moves) > 0

    def player_coord(self, player) -> Tuple[int, int]:
        """
        Versión: 1.0
        Descripción:
            Obtiene la posición actual del jugador.

        Parámetros de entrada:
            - player (int): Identificador del jugador (actualmente soporta 1 para el agente).

        Salida:
            - Tuple[int, int]: Coordenadas (fila, columna) del jugador.

        Notas:
            - Actualmente, retorna siempre la posición del agente.
        """
        return self.agente

    def sucesores(self, mov: Tuple[int, int], player: int, hoyo: Tuple[int, int]) -> 'Tablerowumpus':
        """
        Versión: 1.0
        Descripción:
            Genera el siguiente estado del juego después de que un jugador realice un movimiento. Si el jugador es el agente, se mueve a la nueva posición. Si el jugador es el hoyo, se mueve el hoyo.

        Parámetros de entrada:
            mov (Tuple[int, int]): La coordenada de destino hacia la que el jugador desea moverse.
            player (int): El identificador del jugador que realiza el movimiento (1 para el agente, 2 para el hoyo).
            hoyo (Tuple[int, int]): Las coordenadas del hoyo en caso de que el jugador sea el hoyo.

        Salida:
            Tablerowumpus: Un nuevo estado del juego después de realizar el movimiento.

        Notas:
            - Si el jugador es el agente (player == 1), se mueve utilizando la función `mover`.
            - Si el jugador es el hoyo (player == 2), se mueve utilizando la función `mover_hoyo`.
        """
        new_state = self.getMatrix()  # Crea una copia de la matriz actual
        if player == 1:  # Si el jugador es el agente
            new_state = self.mover(mov)  # Llama a la función mover
        elif player == 2:  # Si el jugador es el hoyo
            new_state = self.mover_hoyo(mov, hoyo)  # Llama a la función mover_hoyo
        return new_state

    def mover(self, coord: Tuple[int, int]) -> 'Tablerowumpus':
        """
        Versión: 1.0
        Descripción:
            Realiza el movimiento del agente a una nueva coordenada. El agente se mueve eliminando su antigua posición y colocándose en la nueva.

        Parámetros de entrada:
            coord (Tuple[int, int]): La nueva coordenada a la que el agente se moverá.

        Salida:
            Tablerowumpus: Un nuevo estado del juego después de mover al agente.

        Notas:
            - Utiliza la función `quitar_tile` para quitar al agente de su antigua posición.
            - Utiliza la función `mover_tile` para mover al agente a la nueva posición.
        """
        new_state = deepcopy(self)  # Crea una copia profunda del estado actual
        antigua_coord = self.agente  # Guarda la posición actual del agente
        new_state.quitar_tile(antigua_coord[0], antigua_coord[1], 1)  # Elimina el agente de la antigua posición
        new_state.mover_tile(coord[0], coord[1], 1)  # Mueve al agente a la nueva posición
        return new_state

    def mover_hoyo(self, coord: Tuple[int, int], antigua_coord: Tuple[int, int]) -> 'Tablerowumpus':
        """
        Versión: 1.0
        Descripción:
            Mueve el hoyo a una nueva posición y ajusta las celdas circundantes según sea necesario (mueve los valores de brisa).

        Parámetros de entrada:
            coord (Tuple[int, int]): La nueva coordenada a la que se moverá el hoyo.
            antigua_coord (Tuple[int, int]): Las coordenadas anteriores del hoyo.

        Salida:
            Tablerowumpus: Un nuevo estado del juego después de mover el hoyo.

        Notas:
            - El hoyo se mueve y se actualizan las celdas vecinas, eliminando las brisas en las celdas que eran vecinas del antiguo hoyo y añadiéndolas en las celdas vecinas del nuevo hoyo.
            - Utiliza las funciones `obtener_vecinos`, `quitar_tile`, y `mover_tile` para modificar el estado.
        """
        new_state = deepcopy(self)  # Crea una copia profunda del estado actual
        vecinos = self.obtener_vecinos(antigua_coord)  # Obtiene las celdas vecinas del antiguo hoyo
        for i in vecinos:  # Elimina las brisas de las celdas vecinas del antiguo hoyo
            new_state.quitar_tile(i[0], i[1], 5)
        new_state.quitar_tile(antigua_coord[0], antigua_coord[1], 6)  # Elimina el hoyo de su antigua posición
        vecinos1 = self.obtener_vecinos(coord)  # Obtiene las celdas vecinas del nuevo hoyo
        for i in vecinos1:  # Añade las brisas a las celdas vecinas del nuevo hoyo
            new_state.mover_tile(i[0], i[1], 5)
        new_state.mover_tile(coord[0], coord[1], 6)  # Mueve el hoyo a la nueva posición
        new_state.comprobar_brisas()  # Revisa y actualiza las celdas con brisas si es necesario
        return new_state

    def mover_hoyos(self) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        """
        Versión: 1.0
        Descripción:
            Encuentra todas las coordenadas válidas para mover los hoyos y retorna un movimiento aleatorio
            junto con la posición original del hoyo.

        Parámetros de entrada:
            Ninguno.

        Salida:
            Tuple[Tuple[int, int], Tuple[int, int]]: Devuelve una tupla con dos coordenadas:
            - La nueva posición a la que se moverá un hoyo.
            - La posición original del hoyo.

        Notas:
            - La función asegura que los hoyos no se muevan a posiciones ocupadas por el agente, el Wumpus, ni el oro.
            - Los movimientos de los hoyos son aleatorios entre los movimientos válidos disponibles.
        """
        coords_hoyos = self.get_hoyo()  # Obtener las posiciones actuales de los hoyos
        movimientos_validos = []  # Lista para almacenar los movimientos válidos
        movimientos_posibles = []  # Lista para almacenar los movimientos posibles
        for coord in coords_hoyos:
            moves_available = self.getAvailableMovesForMin(coord[0], coord[1])  # Movimientos posibles
            for x in moves_available:
                # Verificar que el movimiento sea válido
                if x != self.wumpus and x != self.agente and x != self.oro and x not in coords_hoyos:
                    vecinos_hoyo = self.obtener_vecinos(x)
                    movimientos_posibles.append((x, coord))
                    if all(v != self.wumpus and v != self.agente and v != self.oro for v in vecinos_hoyo):
                        movimientos_validos.append((x, coord))  # Guardar la nueva posición y la posición original

        # Seleccionar un movimiento aleatorio de los válidos
        #if movimientos_validos:
        return random.choice(movimientos_validos)
       # else:
       #     return random.choice(movimientos_posibles)


    def comprobar_brisas(self):
        """
        Versión: 1.0
        Descripción:
            Revisa todas las celdas de la matriz y, si una celda contiene un hoyo (valores 6, 7, 8, 15), coloca una brisa (valor 5) en las celdas vecinas.

        Parámetros de entrada:
            Ninguno.

        Salida:
            Ninguno.

        Notas:
            - Recorre toda la matriz buscando hoyos y, cuando encuentra uno, coloca brisas en las celdas vecinas.
        """
        valores = {6, 7, 8, 15}  # Los valores que representan un hoyo
        coords = []  # Lista para almacenar las coordenadas de los hoyos
        for i in range(len(self.matrix)):
            for j in range(len(self.matrix[0])):
                if self.matrix[i][j] in valores:  # Si la celda contiene un hoyo
                    coord = (i, j)
                    coords.append(coord)  # Añade la coordenada del hoyo a la lista

        # Para cada hoyo encontrado, coloca brisas en las celdas vecinas
        for y in coords:
            vecinos = self.obtener_vecinos(y)  # Obtiene los vecinos de cada hoyo
            for k in vecinos:
                coord1 = (k[0], k[1])
                if self.matrix[k[0]][k[1]] == 0 or self.matrix[k[0]][k[1]] == 6:
                    self.mover_tile(k[0], k[1], 5)  # Coloca una brisa en la celda vecina



    def mover_tile(self, row: int, col: int, tile: int):
        """
        Versión: 1.0
        Descripción:
            Modifica el valor de una casilla en la matriz del tablero, representando la ubicación de un objeto o efecto.

        Parámetros de entrada:
            - row (int): Fila de la casilla a modificar.
            - col (int): Columna de la casilla a modificar.
            - val (int): Valor que se asignará a la casilla.

        Salida:
            Ninguno. Modifica directamente el contenido de la matriz del tablero.

        Notas:
            - La matriz debe estar previamente inicializada.
        """
        current_value = self.matrix[row][col]
        if current_value == 0:  # Si la celda está vacía
            self.matrix[row][col] = tile
        else:
            new_value = self.actualizar_celda(current_value, tile) # Determina el nuevo valor
            self.matrix[row][col] = new_value  # Resetea la celda con el nuevo valor
        if tile == 1: # Si el valor es 1, actualiza la posición del agente
            self.agente = (row, col)

    def quitar_tile(self, row: int, col: int, tile: int):
        """
        Versión: 1.0
        Descripción:
            Cambia el valor de una celda de la matriz a un nuevo valor especificado por `tile`. Si la celda está vacía, se coloca el nuevo valor, de lo contrario se realiza un reset según las reglas de combinación de celdas.

        Parámetros de entrada:
            row (int): La fila de la celda a modificar.
            col (int): La columna de la celda a modificar.
            tile (int): El nuevo valor que se asignará a la celda.

        Salida:
            Ninguno.

        Notas:
            - Si la celda está vacía (valor 0), se coloca directamente el nuevo valor.
            - Si la celda no está vacía, se usa la función `resetear_celda` para determinar el nuevo valor basado en el valor actual de la celda.
            - Si el valor `tile` es 1, también se actualiza la posición del agente.
        """
        current_value = self.matrix[row][col]
        if current_value == 0:  # Si la celda está vacía
            self.matrix[row][col] = tile
        else:
            new_value = self.resetear_celda(current_value, tile)  # Determina el nuevo valor
            self.matrix[row][col] = new_value  # Actualiza la celda con el nuevo valor
        if tile == 1:  # Si el valor es 1, actualiza la posición del agente
            self.agente = (row, col)


    def print_estado_terminal(self):
        """
        Versión: 1.0
        Descripción:
            Verifica si el agente ha alcanzado un estado terminal del juego, como ganar o perder. Imprime el resultado del juego si el estado es terminal.

        Parámetros de entrada:
            Ninguno.

        Salida:
            bool: `True` si el juego ha alcanzado un estado terminal, `False` en caso contrario.

        Notas:
            - El juego termina si el agente alcanza el oro, es atrapado por el Wumpus o cae en dos hoyos.
        """
        global count_hoyos  # Declaramos count_hoyos como global
        if self.agente == self.oro:  # Si el agente ha alcanzado el oro
            print("Has encontrado el oro")
            return True  # Ha ganado el juego

        if self.agente == self.wumpus:  # Si el agente ha sido atrapado por el Wumpus
            print("El Wumpus te ha comido")
            return True  # Ha perdido el juego al ser atrapado

        # Verificamos si el agente está en una de las coordenadas de hoyos
        if self.matrix[self.agente[0]][self.agente[1]] == 7 or self.matrix[self.agente[0]][self.agente[1]] == 8:
            if count_hoyos >= 2:  # Si el agente ha caído en dos hoyos
                print("Has caído en dos hoyos.")
                return True  # Ha perdido el juego
        return False  # Si no es un estado terminal


    def evaluar_estado_terminal_minmax(self) -> bool:
        """
        Versión: 1.0
        Descripción:
            Verifica si el juego ha terminado, ya sea por que el agente ha ganado o ha sido devorado por el Wumpus.

        Parámetros de entrada:
            Ninguno.

        Salida:
            - bool: Retorna True si el juego ha terminado, de lo contrario, False.

        Notas:
            - El juego termina si el agente obtiene el oro o si el agente es devorado por el Wumpus.
        """
        global count_hoyos  # Declaramos count_hoyos como global
        if self.agente == self.oro:
            return True  # Ha encontrado el oro
        if self.agente == self.wumpus:
            return True  # Ha sido atrapado por el Wumpus
        if count_hoyos >= 2:
            return True  # Ha caído en dos hoyos


In [3]:


def minimax(estado: 'Tablerowumpus', current_level: int, depth: int, player: int,
            alpha: float, beta: float, coord: Tuple[int, int], stop: bool) -> Tuple[int, 'Tablerowumpus', Tuple[int,int]]:
    """
    Versión: 1.0
    Descripción:
        Implementa el algoritmo Minimax con poda alfa-beta para evaluar el mejor movimiento en un estado dado.

    Parámetros de entrada:
        estado ('Tablerowumpus'): El estado actual del juego.
        current_level (int): Nivel actual de profundidad en el árbol de búsqueda.
        depth (int): Profundidad máxima de búsqueda.
        player (int): El jugador actual (1 para Max, 2 para Min).
        alpha (float): Valor de alfa para poda.
        beta (float): Valor de beta para poda.
        coord (Tuple[int, int]): Coordenada actual del jugador.
        stop (bool): Indicador para detener la búsqueda si es necesario.

    Salida:
        Tuple[int, 'Tablerowumpus', Tuple[int, int]]: Devuelve una tupla con:
        - La evaluación del estado.
        - El nuevo estado después del movimiento.
        - La coordenada del mejor movimiento.

    Notas:
        - Utiliza el algoritmo Minimax para encontrar el mejor movimiento, optimizado con poda alfa-beta.
        - Si se alcanza la profundidad máxima o un estado terminal, se evalúa el estado.
        - Los jugadores Max (1) y Min (2) realizan movimientos alternados.
    """
    x, hoyo = estado.mover_hoyos()
    movimiento_actual = None
    # Generamos los sucesores
    if estado.moveCanBeMade(player, coord):
        if player == 1:
            moves_available = estado.getAvailableMovesForMax(coord[0], coord[1])
        elif player == 2:
            moves_available = estado.getAvailableMovesForMin(hoyo[0], hoyo[1])
    else:
        moves_available = []

    successorMatrices = []

    for x in moves_available:
        new_state = deepcopy(estado)
        # Llamada a sucesores con el conjunto de estados visitados
        if player == 1:
            sucesor = new_state.sucesores(x, player, None)
        else:
            sucesor = new_state.sucesores(x, player, hoyo)

        # Si el sucesor es válido, lo agregamos a la lista
        if sucesor is not None:
            successorMatrices.append(sucesor)

    if depth == 0 or estado.evaluar_estado_terminal_minmax() or stop:
        return estado.utility(estado.agente, estado.oro), estado, coord

    best_move = moves_available[0]

    best_state = estado  # Inicializamos con el estado actual

    if player == 1:  # Jugador Max
        max_eval = -float('inf')
        for idx, move in enumerate(moves_available):
            eval, new_state, mov = minimax(successorMatrices[idx], current_level + 1, depth - 1, player, alpha, beta, move, stop)
            movimiento_actual = move
            if eval > max_eval:
                max_eval = eval
                best_state = new_state
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                stop = True
                break
        return max_eval, best_state, best_move

    elif player == 2:
        min_eval = float('inf')
        for idx, move in enumerate(moves_available):
            eval, new_state, _ = minimax(successorMatrices[idx], current_level + 1, depth - 1, player, alpha, beta, move, stop)
            movimiento_actual = move
            if eval < min_eval:
                min_eval = eval
                best_state = new_state
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                stop = True
                break
        return min_eval, best_state, best_move

def performActionMinMax(estado: 'Tablerowumpus', player: int) -> Tuple[int, 'Tablerowumpus', Tuple[int, int]]:
    """
    Versión: 1.0
    Descripción:
        Realiza una acción basada en el algoritmo Minimax para el jugador dado. Evalúa el mejor movimiento usando Minimax.

    Parámetros de entrada:
        estado ('Tablerowumpus'): El estado actual del juego.
        player (int): El jugador actual (1 o 2).

    Salida:
        Tuple[int, 'Tablerowumpus', Tuple[int, int]]: Devuelve una tupla con:
        - El valor del estado evaluado.
        - El nuevo estado después de la acción.
        - La coordenada del mejor movimiento.

    Notas:
        - Llama a la función `minimax` para evaluar el mejor movimiento en el estado actual.
        - Actualiza la lista de posiciones visitadas para el jugador.
    """
    global new_visited_positions
    profundidad = 2
    mejor_valor = -float('inf') if player == 1 else float('inf')
    mejor_movimiento = None
    mejor_matriz = None
    stop = False

    if len(new_visited_positions) == 0:
        new_visited_positions[estado.agente] = 'v'

    posicion_actual = estado.player_coord(player)
    valor, matriz_resultante, mejor_movimiento = minimax(estado, 0, profundidad - 1, player, -float('inf'), float('inf'), posicion_actual, stop)

    if (player == 1 and valor > mejor_valor) or (player == 2 and valor < mejor_valor):
        mejor_valor = valor
        mejor_matriz = matriz_resultante

    new_visited_positions[mejor_movimiento] = 'v'

    return mejor_matriz, estado, mejor_movimiento



def AIAction(state: 'Tablerowumpus', player: int) -> 'Tablerowumpus':
    """
    Versión: 1.0
    Descripción:
        Realiza una acción de IA basada en el jugador. Si el jugador es el agente, realiza una acción de Minimax.
        Si es el jugador de los hoyos, mueve un hoyo y evalúa el nuevo estado.

    Parámetros de entrada:
        state ('Tablerowumpus'): El estado actual del juego.
        player (int): El jugador actual (1 o 2).

    Salida:
        'Tablerowumpus': El nuevo estado del juego después de realizar la acción.

    Notas:
        - Si el jugador es el agente (player 1), se realiza una acción de Minimax.
        - Si el jugador es el jugador de los hoyos (player 2), se mueve un hoyo.
        - La función verifica si el agente ha alcanzado una posición terminal, como la victoria o la derrota.
    """
    global AIReadyToMove
    global count_hoyos

    if player == 1:
        mejor_matriz, _, _ = performActionMinMax(state, player)
    else:
        x, hoyo = state.mover_hoyos()
        mejor_matriz = state.sucesores(x,player,hoyo)

    if mejor_matriz.matrix[mejor_matriz.agente[0]][mejor_matriz.agente[1]] in (7, 8):
        count_hoyos += 0.5
        print("total: ", count_hoyos)

    AIReadyToMove = False
    return mejor_matriz


In [4]:
def menu_configuracion():
    """
    Versión: 1.0
    Descripción:
        Muestra un menú para que el usuario elija entre jugar con un tablero por defecto o crear uno modificado.

    Parámetros de entrada:
        Ninguno.

    Salida:
        Tuple[int, Optional[int], Tuple[Optional[int, int], Optional[int, int]]]:
        Devuelve el tamaño del tablero, el número de hoyos y las coordenadas fijas para el oro y el Wumpus.

    Notas:
        - Si el usuario elige la opción 1, se usa la configuración predeterminada.
        - Si elige la opción 2, permite personalizar el tablero.
        - Si se elige la opción 3, el programa se cierra.
    """
    while True:
        print("\n--- Menú Principal ---")
        print("1. Jugar con el tablero por defecto")
        print("2. Crear un tablero modificado")

        try:
            opcion = int(input("Selecciona una opción: "))
            if opcion == 1:
                tamanyo = 8
                num_hoyos = None
                coord_oro = (None,None)
                coord_wumpus = (None,None)
                print("\nUsando configuración por defecto.")
                break
            elif opcion == 2:
                tamanyo = pedir_tamanyo_tablero()
                num_hoyos = pedir_num_hoyos()
                coord_oro = pedir_coordenada_fija("oro", tamanyo)
                coord_wumpus = pedir_coordenada_fija("wumpus", tamanyo)
                print("\nConfiguración personalizada establecida.")
                break
            else:
                print("Opción inválida. Por favor, selecciona entre 1, 2 y 3.")
        except ValueError:
            print("Entrada inválida. Por favor, introduce un número.")

    return tamanyo, num_hoyos, coord_oro, coord_wumpus



def pedir_tamanyo_tablero():
    """
    Versión: 1.0
    Descripción:
        Pide al usuario el tamaño del tablero, asegurándose de que esté entre 6 y 8.

    Parámetros de entrada:
        Ninguno.

    Salida:
        int: El tamaño del tablero (un número entre 6 y 8).

    Notas:
        - Si el usuario introduce un valor fuera del rango permitido o un valor no numérico, se vuelve a pedir la entrada.
    """
    while True:
        try:
            tamanyo = int(input("Introduce el tamaño del tablero (mínimo 6, máximo 8): "))
            if 6 <= tamanyo <= 8:
                return tamanyo
            else:
                print("El tamaño debe estar entre 6 y 8.")
        except ValueError:
            print("Entrada inválida. Por favor, introduce un número.")



def pedir_num_hoyos():
    """
    Versión: 1.0
    Descripción:
        Pide al usuario un número fijo de hoyos, con un máximo de 5 hoyos.

    Parámetros de entrada:
        Ninguno.

    Salida:
        int: El número de hoyos (un número entre 1 y 5).

    Notas:
        - Si el número de hoyos es inválido o no está en el rango permitido, vuelve a pedir la entrada.
    """
    while True:
        try:
            num_hoyos = int(input("Introduce el número de hoyos (máximo 5): "))
            if 1 <= num_hoyos <= 5:
                return num_hoyos
            else:
                print("El número de hoyos debe estar entre 1 y 5.")
        except ValueError:
            print("Entrada inválida. Por favor, introduce un número.")



def pedir_coordenada_fija(elemento, tamanyo):
    """
    Versión: 1.0
    Descripción:
        Pide al usuario una coordenada fija para un elemento (oro o Wumpus) en el tablero.
        Si el usuario no introduce ninguna coordenada, se selecciona una aleatoria.

    Parámetros de entrada:
        elemento (str): El nombre del elemento para el cual se solicita la coordenada ("oro" o "wumpus").
        tamanyo (int): El tamaño del tablero para validar las coordenadas.

    Salida:
        Tuple[Optional[int, int], Optional[int, int]]: La coordenada fija para el elemento. Si se deja vacío, devuelve `(None, None)` para seleccionar aleatoriamente.

    Notas:
        - Si el usuario introduce una coordenada fuera del rango o en una posición restringida, se le solicita nuevamente.
    """
    while True:
        entrada = input(f"Introduce la coordenada fija para el {elemento} en el formato 'x,y' (o presiona Enter para aleatorio): ")
        if entrada.strip() == "":
            return None, None  # Coordenada aleatoria
        try:
            x, y = map(int, entrada.split(","))
            if 0 <= x < tamanyo and 0 <= y < tamanyo:
                # Validar que no esté en posiciones restringidas
                if (x, y) not in [(6, 0), (6, 1), (7, 0), (7, 1)]:
                    return x, y
                else:
                    print(f"La coordenada ({x}, {y}) no es válida. No puede estar en (6, 0), (6, 1), (7, 0) o (7, 1).")
            else:
                print(f"Las coordenadas deben estar entre 0 y {tamanyo - 1}.")
        except ValueError:
            print("Entrada inválida. Por favor, introduce dos números separados por una coma.")



In [5]:

def main():
    """
    Versión: 1.0
    Descripción:
        Función principal que coordina el flujo del juego.
        Lee la configuración del tablero, inicializa el estado del juego y ejecuta las acciones de los jugadores.

    Parámetros de entrada:
        Ninguno.

    Salida:
        Ninguno.

    Notas:
        - La función organiza la interacción entre las diferentes funciones del programa, como la configuración y la ejecución del juego.
        - Los jugadores (agente y hoyos) se alternan en cada turno.
        - Después de cada turno, el estado del tablero se actualiza y muestra al usuario.
    """

    # Mostrar menú y obtener configuración
    configuracion = menu_configuracion()
    if configuracion is None:
        return  # El usuario decidió salir del programa

    tamanyo, num_hoyos, coord_oro, coord_wumpus = configuracion
    initial_matrix = [[0 for _ in range(tamanyo)] for _ in range(tamanyo)]

    # Crear el tablero con los parámetros configurados
    tablero = Tablerowumpus(initial_matrix, num_hoyos, coord_oro, coord_wumpus)

    display(tablero.get_html())
    time.sleep(3)
    agente = tablero.agente  # Asumiendo que `agente` se define en `Tablerowumpus`
    estado_inicial = tablero
    jugador = 1  # Comienza el jugador 1 (agente)

    while not estado_inicial.evaluar_estado_terminal_minmax():

        if jugador == 1:
            estado_inicial = AIAction(estado_inicial, jugador)
            #estado_inicial.evaluar_estado_terminal()
        elif jugador == 2:
            estado_inicial = AIAction(estado_inicial, jugador)
            #estado_inicial.evaluar_estado_terminal()

        # Limpiar la salida previa en Jupyter
        clear_output(wait=True)

        # Mostrar el nuevo estado del tablero
        display(estado_inicial.get_html())
        #print("hoyos: ", count_hoyos)

        # Espera de 5 segundos antes de la siguiente iteracion
        time.sleep(1)

        # Cambiar de jugador
        jugador = 2 if jugador == 1 else 1

    if estado_inicial.evaluar_estado_terminal_minmax():
        estado_inicial.print_estado_terminal()

if __name__ == "__main__":
    main()

,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,
,,,,,,,


Has encontrado el oro
